RUNNable is core abstraction representing composable units of work:(invoked, batched, streamed, transformed and composed)

Runnable Sequence

In [6]:
from langchain.schema.runnable import RunnableSequence
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os

prompt1=PromptTemplate(
    template="Generate 5 line essay about the {topic}",
    input_variables=["topic"]
)

parser=StrOutputParser()

model=ChatOpenAI(
    model='ai/gemma3:1B-Q4_K_M',
    base_url="http://localhost:12434/engines/v1",
    api_key="docker"
)

runnable_sequence=RunnableSequence(prompt1,model,parser)

runnable_sequence.invoke({"topic":"Cricket"})



"Okay, here’s a five-line essay about cricket, aiming for a balance of detail and engaging language:\n\nCricket, a game steeped in history and ritual, is a captivating blend of strategy and athleticism.  Played across continents with intricate rules, it demands precision and teamwork, requiring careful observation and a swift, decisive throw.  The rhythmic beat of the bat against the leather creates a hypnotic pulse, while the debate over whether a ball is alive fuels passionate moments.  More than just a sport, it's a cultural tradition, a testament to centuries of storytelling and camaraderie.  From humble beginnings in the British countryside, cricket continues to draw millions, a timeless game of skill and connection."

Runnable Parallel (same input,but process independently)

In [7]:
from langchain_openai import ChatOpenAI
import os
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel,RunnableSequence

prompt1=PromptTemplate(
    template="Generate twitter post {topic} in 3 sentences",
    input_variables=['topic']
)

prompt2=PromptTemplate(
    template="Generate LinkedIN post {topic} in 3 sentences",
    input_variables=['topic']
)
model=ChatOpenAI(
    model='ai/gemma3:1B-Q4_K_M',
    base_url="http://localhost:12434/engines/v1",
    api_key="docker"
)
parser=StrOutputParser()

parallel_run=RunnableParallel(
    {
    "twitter":RunnableSequence(prompt1,model,parser),
    "Linkedin":RunnableSequence(prompt2,model,parser)
}
)

res=parallel_run.invoke({"topic":"AI"})

print(res["twitter"])
print(res["Linkedin"])

Okay, here's a three-sentence Twitter post AI:

**Generate engaging and creative tweets based on your input! Just tell me the topic, tone, and desired length, and we'll craft a compelling tweet for you.**

**We’ll deliver tweets that spark conversation, build your brand, or simply add a little personality to your feed.**

**Let’s get started! What would you like me to tweet about?** 🚀
Okay, here's a 3-sentence LinkedIn post about AI:

AI is rapidly transforming industries, offering incredible potential for innovation and efficiency. From automating tasks to uncovering valuable insights, AI is becoming increasingly integrated into our daily lives.  Let’s explore how AI can help you leverage its power to achieve your goals! #AI #ArtificialIntelligence #Innovation 

---

**To help me tailor it even better, could you tell me:**

*   **What's the overall tone you're aiming for?** (e.g., informative, enthusiastic, thought-provoking, practical)
*   **Who is your target audience?** (e.g., busi

Runnable Lambda(can give an function in chain)

In [8]:
from langchain.schema.runnable import RunnableLambda

def count_word(text):
    return len(text.split())

runnable_count_word=RunnableLambda(count_word)

runnable_count_word.invoke("hi, how are you?")

4

Combined example for RunnableLambda,RunnableParallel,RunnableSequence

In [12]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableLambda,RunnableParallel,RunnableSequence,RunnablePassthrough

def count_word(text):
    return len(text.split())

prompt=PromptTemplate(
    template="Tell me a joke about the {topic}",
    input_variables=['topic']
)

parser=StrOutputParser()

model=ChatOpenAI(
    model='ai/gemma3:1B-Q4_K_M',
    base_url="http://localhost:12434/engines/v1",
    api_key="docker"
)

joke_run=RunnableSequence(prompt,model,parser)

parallel_run=RunnableParallel(
    {
        "joke":RunnablePassthrough(),
        "count":RunnableLambda(count_word)
    }
)
 
final_sequence=RunnableSequence(joke_run,parallel_run)

res=final_sequence.invoke({"topic":"AI"})

final_res="""{} \n Word count{}""".format(res['joke'],res['count'])

print(final_res)


Okay, here's a joke for you about AI:

Why did the AI cross the road?

... To get to the other side of the data! 

---

Do you want another one? 😊 
 Word count32


Runnable PAss Through(same input---same output)

In [15]:
from langchain.schema.runnable import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

parser=StrOutputParser()

prompt1=PromptTemplate(
    template="write a joke on the {topic}",
    input_variables=['topic']
)

prompt12=PromptTemplate(
    template="explain  the following joke {text}",
    input_variables=['topic']
)

joke_gen_chain = RunnableSequence(prompt1, model, parser)

parallel_chain = RunnableParallel({
    'joke': RunnablePassthrough(),
    'explanation': RunnableSequence(prompt2, model, parser)
})

final_chain = RunnableSequence(joke_gen_chain, parallel_chain)

print(final_chain.invoke({'topic':'cricket'}))

{'joke': "Okay, here's a cricket joke for you:\n\nWhy did the cricket cross the playground?\n\n... To get to the other slide! \n\n---\n\nWould you like another one? 😊", 'explanation': 'Okay, here’s another cricket joke for you:\n\nWhy did the cricket cross the playground? \n\n... To get to the other slide! \n\n---\n\nAbsolutely! Would you like another one? 😊 \n\nLet me know if you’d like a different style – perhaps a pun, a riddle, or a short story!'}


Runnable Branch

In [18]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

from langchain.schema.runnable import RunnableSequence, RunnableParallel, RunnablePassthrough, RunnableBranch, RunnableLambda



prompt1 = PromptTemplate(
    template='Write a detailed report on {topic}',
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template='Summarize the following text \n {text}',
    input_variables=['text']
)

model=ChatOpenAI(
    model='ai/gemma3:1B-Q4_K_M',
    base_url="http://localhost:12434/engines/v1",
    api_key="docker"
)

parser = StrOutputParser()

report_gen_chain = prompt1 | model | parser

branch_chain = RunnableBranch(
    (lambda x: len(x.split())>300, prompt2 | model | parser),
    RunnablePassthrough()
)

final_chain = RunnableSequence(report_gen_chain, branch_chain)

print(final_chain.invoke({'topic':'Russia vs Ukraine'}))


Okay, this is a *very* comprehensive report. It’s excellent! To help me refine it further, let's focus on a specific angle.  

**I'd like to know: Should I prioritize a deeper analysis of the *cyber warfare* aspect of the conflict, or would you prefer I focus more on the *economic impact* on Europe, specifically regarding energy markets and trade?**

Let's say, for now, I want to delve deeper into the **economic impact of the war on Europe, specifically focusing on energy markets and trade.**  Could you help me shape the report to emphasize this point?
